In [4]:
import pandas as pd
import os

dataset_name = 'en'

for seed in range(5):
    ans = []
    for turn in range(5):
        if os.path.exists(f'results/{dataset_name}/seed{seed}/result{turn}.parquet'):
            temp = pd.read_parquet(f'results/{dataset_name}/seed{seed}/result{turn}.parquet')
            temp.columns = ['job_title', 'job_description', f'task_seed{seed}']
            temp[f'task_seed{seed}'] = temp[f'task_seed{seed}'].apply(lambda x: [x[i] for i in range(len(x)) if x[i] not in x[:i]])
            ans.append(temp)
    if len(ans) == 0:
        break
    ans = pd.concat(ans)
    if seed == 0:
        final_task = ans
    else:
        final_task = pd.merge(final_task, ans, on=['job_title', 'job_description'])
final_task

,job_title,job_description,task_seed0
0,Senior Podcast Producer,"Lead production on assigned series, directing ...","[Explain use of products or services., Monitor..."
1,QC Biochemistry Supervisor,"hiring, performance management and supporting ...","[Read materials to determine needed actions., ..."
2,"NDT Assistant - Entry Level, Rapid Growth",Set up and utilize Nondestructive Test equipme...,"[Collect samples for analysis or testing., Ope..."
3,Field Construction Coordinator,Assisting Construction and Engineering teams w...,"[Search files, databases or reference material..."
4,Legal Executive Assistant,Conduct legal research and gather relevant inf...,"[Communicate with government agencies., Delive..."
...,...,...,...
10,Sr React Frontend Lead Developer -onsite Job o...,UI Development and Performance OptimizationAs ...,[Prepare drawings or diagrams of products or s...
11,Marketing Specialist I,Ensure all deployed emails have assigned sende...,"[Transport clients to appointments., Manage in..."
12,Medical Assistant- Lone Tree Family Practice,Completes intake by welcoming patients and pre...,[Present social services program information t...
13,Registered Nurse (RN) - Pediatric Emergency De...,"Sedatives and Paralytics, Analgesics, Adrenerg...",[Administer anesthetics or sedatives to contro...


In [5]:
origin_df = pd.read_csv(f'../step3-bert_recall/CuBERT/results/{dataset_name}/2e-05_32_0.8/{dataset_name}.csv', sep='\t')
origin_df['sim'] = origin_df.apply(lambda x: sum([x[f"task{i}_sim"] for i in range(100)]), axis=1)
origin_df = origin_df[['job_title', 'job_description', 'sim']]
final_task = pd.merge(final_task, origin_df, on=['job_title', 'job_description'])
final_task

,job_title,job_description,task_seed0,sim
0,Senior Podcast Producer,"Lead production on assigned series, directing ...","[Explain use of products or services., Monitor...",99.568766
1,QC Biochemistry Supervisor,"hiring, performance management and supporting ...","[Read materials to determine needed actions., ...",99.497819
2,"NDT Assistant - Entry Level, Rapid Growth",Set up and utilize Nondestructive Test equipme...,"[Collect samples for analysis or testing., Ope...",98.452221
3,Field Construction Coordinator,Assisting Construction and Engineering teams w...,"[Search files, databases or reference material...",99.579612
4,Legal Executive Assistant,Conduct legal research and gather relevant inf...,"[Communicate with government agencies., Delive...",99.642757
...,...,...,...,...
7772,Sr React Frontend Lead Developer -onsite Job o...,UI Development and Performance OptimizationAs ...,[Prepare drawings or diagrams of products or s...,99.280104
7773,Marketing Specialist I,Ensure all deployed emails have assigned sende...,"[Transport clients to appointments., Manage in...",99.537967
7774,Medical Assistant- Lone Tree Family Practice,Completes intake by welcoming patients and pre...,[Present social services program information t...,99.015374
7775,Registered Nurse (RN) - Pediatric Emergency De...,"Sedatives and Paralytics, Analgesics, Adrenerg...",[Administer anesthetics or sedatives to contro...,97.835360


In [6]:
from utlis import aggregate_expert_opinions
final_task['task'] = final_task.apply(lambda x: aggregate_expert_opinions([x[f'task_seed{i}'] for i in range(5) if f'task_seed{i}' in x]), axis=1)
final_task['score'] = final_task['task'].apply(lambda x: x[1])
final_task['task'] = final_task['task'].apply(lambda x: x[0])
final_ans = final_task[['job_title', 'job_description', 'task', 'sim', 'score']]
final_ans = final_ans.sort_values(['score', 'sim'], ascending=[False, False])
final_ans.to_excel(f'results/{dataset_name}/{dataset_name}_results.xlsx', index=None)
final_ans

,job_title,job_description,task,sim,score
3375,Wastewater/Maintenance Technician- Valent BioS...,"Operates, maintains, and repairs process units...",Lubricate production equipment.,99.795976,10.0
6172,Floor Finisher,Performs maintenance as required on electric a...,"Clean equipment, parts, or tools to repair or ...",99.795635,10.0
6744,Operator,Clean up work areas Fuel vehicles and equipmen...,"Clean equipment, parts, or tools to repair or ...",99.794594,10.0
3166,Maintenance Mechanic,Will be required to troubleshoot and repair al...,Maintain production or processing equipment.,99.792460,10.0
6653,Industrial Equipment Maintenance Technicians,Perform maintenance and repair to the lift tru...,Repair production equipment or tools.,99.791464,10.0
...,...,...,...,...,...
3038,Registered Nurse -Outpatient Surgery Center Ha...,Protect sedated and anesthetized patients by i...,Assist healthcare practitioners during surgery.,94.893992,10.0
2623,Aesthetics Physician Assistant,"time, permanent roleMonday-Friday, 8am-5pmProv...",Present information to the public.,94.832019,10.0
946,Child Care Bus Driver,Caring! Transport children in a bus or van fro...,Assist disabled or incapacitated individuals.,94.415067,10.0
7737,Sales Associate - Valley Pool & Spa - Seasonal,"Selling skills, personable and friendly - full...",Recommend products or services to customers.,91.505593,10.0
